### Problem 1: Compute the rank 1 constraint system for the following:
```
fn main(x: field, y: field) -> field {
  return 5*x**3 - 4*y**2*x**2 + 13*x*y**2 + x**2 - 10*y
}
```
which is essentially $out = 5x^3 - 4x^2y^2 + 13xy^2 + x^2 - 10y$

We observe that we get 10y for free (1 multiplication per constraint): $out + 10y = 5x^3 - 4x^2y^2 + 13xy^2 - x^2$ 

We then define the constraints as
1. $v_1 = x * x$
2. $v_2 = y * y$
3. $v_3 = v_1 * x$
4. $v_4 = v_1 * v_2$
5. $out - 5v_3 + 4v_4 + v_1 + 10y = 13xv_2$

and the witness $\vec{w} = \begin{bmatrix} 1 \ out \ x \ y \ v_1 \ v_2 \ v_3 \ v_4 \end{bmatrix}$

Now, we transform it into R1CS: $C\vec{w} = A\vec{w} B\vec{w}$

Since we have 3 constraints, so do the matrices:
A is the left hand side of the constraints: $A = 
\begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\\
0 & 0 & 13 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$

B is the right hand side of the constraints: $B = \begin{bmatrix} 
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\ 
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0
\end{bmatrix}$ 

C is the result $C = \begin{bmatrix} 
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\\ 
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\\
0 & 1 & 0 & 10 & 1 & 0 & -5 & 4
\end{bmatrix} $

### Problem 2: Compute the rank 1 constraint system for the following:

```
fn main(x: field, y: field) -> field {
  assert!(y == 0 || y == 1 || y == 2);
  if (y == 0) {
		return x; 
	}
	else if (y == 1) {
	  return x**2;
	} 
	else {
	  return x**3;
	}
}
```

Hint: can you represent the assert statement as a polynomial?

Hint: can you construct a polynomial for the if-else-if-else statement using a more
general construction than the if-else we showed in class?

The assert function is essentially $(y-0)(y-1)(y-2) = 0$. To represent the if-else-if-else statement, we can use a construct: $out = ax + bx^2 + cx^3$, where:
- $ a = \frac{(y-1)(y-2)}{(0-1)(0-2)} $ which results in 1 when $y=0$
- $ b = \frac{(y-0)(y-2)}{(1-0)(1-2)} $ which results in 1 when $y=1$
- $ c = \frac{(y-0)(y-1)}{(2-0)(2-1)} $ which results in 1 when $y=2$

The idea is that for a specific y, e.g. $y=1$, we exclude the factor in $(y-0)(y-1)(y-2)$ that results in 0 from the numerator. In this case, we exclude $y-1$. Then, with the remaining factors $(y-0)(y-2)$, we evaluate them at $y=1$.

Thus the final expression is: $out = \frac{(y-1)(y-2)}{2} \cdot x + \frac{(y-0)(y-2)}{-1} \cdot x^2 + \frac{(y-0)(y-1)}{2} \cdot x^3$

which can be rewritten as $2*out = y^2x -3xy + 2x - 2x^2y^2 + 4x^2y + x^3y^2 - x^3y$

The assert function can be rewritten as $y^3 - 3y^2 + 2y = 0$. Let $v_1 = y^2$ we get $3v_1 - 2y = v_1y$

Let $v_2 = xy$. Then $2 * out = v_2y - 3v_2 + 2x - 2v_2^2 + 4v_2x + v_2^2x - v_2x^2$ which is $2 * out = v_2 (y - 2v_2 + 4x + v_2x - x^2) - 3v_2 + 2x$

Let $v_3 = v_2x$. Then $2 * out = v_2 (y - 2v_2 + 4x + v_3 - x^2) - 3v_2 + 2x$

Let $v_4 = y - 2v_2 + 4x + v_3 - x^2$. Then $2 * out + 3v_2 - 2x = v_2 v_4$

To recap
1. $v_1 = y * y$
2. $-2y + 3v_1 = v_1y$
3. $v_2 = x * y$
4. $v_3 = v_2 * x$
5. $4x + y - 2v_2 + v_3 - v_4 = x * x$
7. $2out - 2x + 3v_2 = v_2 v_4$

and the witness $\vec{w} = \begin{bmatrix} 1 \ out \ x \ y \ v_1 \ v_2 \ v_3 \ v_4 \end{bmatrix}$


$A = 
\begin{bmatrix}
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\\ 
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0
\end{bmatrix}
$
$B = 
\begin{bmatrix}
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
$
$C = 
\begin{bmatrix}
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & -2 & 3 & 0 & 0 & 0 \\\ 
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\\
0 & 0 & 4 & 1 & 0 & -2 & 1 & -1 \\\
0 & 2 & -2 & 0 & 0 & 3 & 0 & 0
\end{bmatrix}
$

## Problem 3

In [195]:
# Compute everything in mod prime p
p = 23
def verify_r1cs(A, B, C, w):
    return all(el == 0 for el in np.multiply(A.dot(w) % p, B.dot(w) % p) % p - C.dot(w) % p)

In [196]:
# For Problem 1
import numpy as np
import random

A = np.array([
 [ 0,  0,  1,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  1,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  1,  0,  0,  0],
 [ 0,  0,  0,  0,  1,  0,  0,  0],
 [ 0,  0, 13,  0,  0,  0,  0,  0]
]);
B = np.array([[0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0]]);
C= np.array([[ 0,  0,  0,  0,  1,  0,  0,  0],
 [ 0,  0,  0,  0,  0,  1,  0,  0],
 [ 0,  0,  0,  0,  0,  0,  1,  0],
 [ 0,  0,  0,  0,  0,  0,  0,  1],
 [ 0,  1,  0, 10, 1,  0, -5,  4]]);

x = random.randint(1,1000)
y = random.randint(1,1000)

v1 = x * x % p
v2 = y * y % p
v3 = v1 * x % p
v4 = v1 * v2 % p
out = 5*v3 - 4*v4 - v1 - 10*y + 13*x*v2 % p
w = np.array([1, out, x, y, v1, v2, v3, v4]);

verify_r1cs(A, B, C, w)

True

In [193]:
# For Problem 2
A = np.array([
[0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0]
]);
B = np.array([
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1]
]);
C= np.array([
 [ 0,  0,  0,  0,  1,  0,  0,  0],
 [ 0,  0,  0,  -2, 3,  0,  0,  0],
 [ 0,  0,  0,  0,  0,  1,  0,  0],
 [ 0,  0,  0,  0,  0,  0,  1,  0],
 [ 0,  0, 4, 1,  0,  -2, 1,  -1],
 [ 0,  2, -2,  0,  0,  3,  0,  0]
]);

x = random.randint(1,1000)
y = random.randint(0,2)

half = pow(2,-1,p)
v1 = (y * y) % p
# this is wrong because we're redefining y, perhaps it can only be asserted
# y = ((3*v1 + v1*y) * half) % p 
v2 = (x * y) % p
v3 = (v2 * x) % p
v4 = (4*x + y - 2*v2 + v3 - x * x) % p
out = ((2*x - 3*v2 + v2*v4)*half) % p
w = np.array([1, out, x, y, v1, v2, v3, v4]);

verify_r1cs(A, B, C, w)

True